In [1]:
import phoebe
import numpy as np
import matplotlib.pyplot as plt
import emcee
import sys
%matplotlib inline

In [2]:
b = phoebe.default_binary(contact_binary = True)

In [3]:
#b.add_constraint('semidetached', 'primary')

In [4]:
#b.add_constraint('semidetached', 'secondary')

In [5]:
b['period@orbit'] = 1
b['sma@orbit'] = 1
#b['q@binary'] = 0.44
# b['incl@orbit'] = 83.5
# b['requiv@primary'] = 1.2
# b['requiv@secondary'] = 0.8
#b['teff@primary'] = 5660.
# b['teff@secondary'] = 5500.

In [6]:
#lc = np.loadtxt('data.lc')
v737 = np.loadtxt('v737per.B')
vdata = v737[:,1]
flux = vdata/(-2.5)
flux10 = 10**flux
lc = v737
lc[:,1] = flux10
sigmas = np.diff(flux10,2).std()*np.ones(len(lc))

In [7]:
#b.add_dataset('lc', times=lc[:,0], fluxes=lc[:,1], sigmas=0.05*np.ones(len(lc)), passband='Johnson:B')
b.add_dataset('lc', times=lc[:,0], fluxes=lc[:,1], sigmas=sigmas, passband='Johnson:B')

<ParameterSet: 78 parameters | contexts: dataset, constraint, figure, compute>

In [8]:
phoebe.interactive_checks_off()
phoebe.interactive_constraints_off()
b.set_value_all('irrad_method', 'none')

In [9]:
b.flip_constraint('compute_phases', 'compute_times')
b['compute_phases@lc@dataset'] = np.linspace(-0.5,0.5,len(lc))

In [10]:
def lnprob(x, adjpars, priors):
    #Check to see that all values are within the allowed limits:
    if not np.all([priors[i][0] < x[i] < priors[i][1] for i in range(len(priors))]):
         return -np.inf

    for i in range(len(adjpars)):
        b[adjpars[i]] = x[i]
    
    # Let's assume that our priors are uniform on the range of the physical parameter combinations.
    # This is already handled in Phoebe, which will throw an error if the system is not physical,
    # therefore it's easy to implement the lnprior as =0 when system checks pass and =-inf if they don't.
    # Here we'll 'package' this in a simple try/except statement:
    
    try:
        b.run_compute(irrad_method='none')

        # sum of squares of the residuals
        fluxes_model = b['fluxes@model'].interp_value(times=lc[:,0])
        #lnp = -0.5*np.sum((fluxes_model-b['value@fluxes@dataset'])**2 / b['value@sigmas@dataset']**2)
        mag_model = -2.5*np.log10(fluxes_model) 
        mag_model = mag_model-np.mean(mag_model)
        mag_value = -2.5*np.log10(b['value@fluxes@dataset'])
        mag_value = mag_value-np.mean(mag_value)
        lnp = -0.5*np.sum((mag_model-mag_value)**2 / b['value@sigmas@dataset']**2)
        
        #print('inp is ok!')

    except:
        lnp = -np.inf

    sys.stderr.write("lnp = %e\n" % (lnp))

    return lnp

In [11]:
from scipy.stats import norm
def run(adjpars, priors, nwalkers, niter):
    ndim = len(adjpars)

   

    p0 = np.array([[p[0] + (p[1]-p[0])*np.random.rand() for p in priors] for i in range(nwalkers)])

#     pool = MPIPool()
#     if not pool.is_master():
#         pool.wait()
#         sys.exit(0)

    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=[adjpars, priors])

    pos, prob, state = sampler.run_mcmc(p0, niter)
    
    print(p0)
    
    for i in range(ndim):
        plt.figure()
        y = sampler.flatchain[:,i]
        n, bins, patches = plt.hist(y, 200, density=1, color="b", alpha=0.45)
        plt.title("Dimension {0:d}".format(i))
        
        mu = np.average(y)
        sigma = np.std(y)       
        print ("mu,", "sigma = ", mu, sigma)

        bf = norm.pdf(bins, mu, sigma)
        l = plt.plot(bins, bf, 'k--', linewidth=2.0)
         
    #plt.show()
    
    return pos

#     pool.close()



In [ ]:
adjpars = ['requiv@primary', 'incl@orbit', 'q@binary', 'teff@primary', 'teff@secondary']
priors = [(0.45,0.55), (76.0, 79.0), (0.42, 0.45), (5400, 5800), (5400., 5800.)]
nwalkers = 100

niters = 20
state = None

import time

time1 = time.time()
postion = run(adjpars, priors, nwalkers, niters)
time2 = time.time()

run_time = time2-time1

print(run_time)

100%|██████████| 310/310 [00:10<00:00, 29.11it/s]
lnp = -5.393664e+01
100%|██████████| 310/310 [00:10<00:00, 29.83it/s]
lnp = -1.028628e+02
100%|██████████| 310/310 [00:10<00:00, 30.07it/s]
lnp = -8.763286e+01
100%|██████████| 310/310 [00:10<00:00, 29.97it/s]
lnp = -1.954533e+01
100%|██████████| 310/310 [00:10<00:00, 29.90it/s]
lnp = -1.538106e+02
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 29.69it/s]
lnp = -2.323995e+01
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 29.64it/s]
lnp = -2.581580e+01
100%|██████████| 310/310 [00:10<00:00, 30.17it/s]
lnp = -5.428239e+01
100%|██████████| 310/310 [00:10<00:00, 29.90it/s]
lnp = -1.926007e+01
100%|██████████| 310/310 [00:10<00:00, 29.87it/s]
lnp = -1.580059e+02
100%|██████████| 310/310 [00:10<00:00, 29.64it/s]
lnp = -3.265151e+01
lnp = -inf
lnp = -inf
lnp = -inf
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 28.70it/s]
lnp = -1.658360e+01
100%|██████████| 310/310 [00:10<00:00, 30.05it/s]
lnp = -8.742074e+01
100%|██████████| 310/310

100%|██████████| 310/310 [00:10<00:00, 28.89it/s]
lnp = -3.578091e+01
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 28.57it/s]
lnp = -3.610578e+01
100%|██████████| 310/310 [00:10<00:00, 29.48it/s]
lnp = -7.037073e+01
100%|██████████| 310/310 [00:10<00:00, 28.81it/s]
lnp = -2.669756e+01
100%|██████████| 310/310 [00:10<00:00, 28.63it/s]
lnp = -3.810195e+01
100%|██████████| 310/310 [00:10<00:00, 29.23it/s]
lnp = -7.015212e+01
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 29.56it/s]
lnp = -6.474099e+01
100%|██████████| 310/310 [00:10<00:00, 29.22it/s]
lnp = -1.458419e+02
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 30.19it/s]
lnp = -1.422585e+02
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 28.57it/s]
lnp = -8.812932e+01
100%|██████████| 310/310 [00:10<00:00, 29.50it/s]
lnp = -2.679754e+02
100%|██████████| 310/310 [00:10<00:00, 30.26it/s]
lnp = -6.896631e+01
100%|██████████| 310/310 [00:10<00:00, 29.76it/s]
lnp = -1.586190e+02
100%|██████████| 310/310 [00:10<00:00, 28.73it

In [ ]:

from matplotlib.pyplot import cm 

mod = b
position = postion
times = lc[:,0]
color=cm.rainbow(np.linspace(0,1,nwalkers))

for i,c in zip(range(nwalkers),color):
    
    mod['requiv@primary'] = position[-1-i,0]
    mod['incl@binary@orbit@component'] = position[-1-i,1]
    mod['q@binary'] = position[-1-i,2]
    mod['teff@primary'] = position[-1-i,3]
    mod['teff@secondary'] = position[-1-i,4]
    try:
        mod.run_compute(model='run{}'.format(i))
    except:
        print('it is error')


for i,c in zip(range(nwalkers),color):
    try:
        model = mod['fluxes@run{}'.format(i)].interp_value(times=times)

    #plt.figure(1)
        plt.plot(times,model,c=c)
    except:
        print('it is error')
    
plt.plot(times,lc[:,1],"k.")
plt.xlabel("phases")
plt.ylabel("Flux")


In [ ]:
mod['incl@binary@orbit@component']

In [ ]:
mod['q@binary']

In [ ]:
mod['teff@primary']

In [ ]:
mod['teff@secondary']

In [ ]:
from matplotlib.pyplot import cm 

mod = b
position = postion
times = lc[:,0]
color=cm.rainbow(np.linspace(0,1,nwalkers))

for i,c in zip(range(nwalkers),color):
    
    mod['requiv@primary'] = position[-1-i,0]
    mod['incl@binary@orbit@component'] = position[-1-i,1]
    mod['q@binary'] = position[-1-i,2]
    mod['teff@primary'] = position[-1-i,3]
    mod['teff@secondary'] = position[-1-i,4]
    try:
        mod.run_compute(model='run{}'.format(i))
    except:
        print('it is error')


for i,c in zip(range(nwalkers),color):
    try:
        model = mod['fluxes@run{}'.format(i)].interp_value(times=times)

    #plt.figure(1)
        rmodel = -2.5*np.log10(model)
        resultmodel = rmodel-np.mean(rmodel)
        plt.plot(times,resultmodel,c=c)
    except:
        print('it is error')

lmodel = -2.5*np.log10(lc[:,1])  
lmag = lmodel-np.mean(lmodel)
ax = plt.gca()
plt.plot(times,lmag,"k.")
plt.xlabel("phases")
plt.ylabel("Mag")
ax.yaxis.set_ticks_position('left')
ax.invert_yaxis()